<a href="https://colab.research.google.com/github/ayanbabusona/DeepLearning/blob/main/Genetic_Algorithm_to_optimize_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use of Genetic Algorithm to optimize **LSTM**

To begin with let us first implement the genetic algorithm step by step and solve a basic problem with it and then we will optimize LSTM using this implemented Genetic Algorithm.

## Guess the Word
Here the genetic algorithms will be used to guess a word. The genetic algorithm will know the number of letters in the word and will guess those letters until it finds the right answer. It has been decided to represent the genes as a single alphanumeric character; strings of these characters thus constitute a chromosome. And our fitness function is the sum of the characters matching in the individual and the right word.

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
#dependency installation
!pip install deap

As the first step, The modules are imported. The string module and the random module would be used to generate random characters from (a—z, A—Z, and 0—9). From the DEAP module, creator, base, and tools are essentially needed to implement this algorithm.

In [23]:
import string
import random
from deap import base, creator, tools

Let's create a class which inherits the deap.base module and define the type of optimization (i.e. maximization, minimization). Maximization represents with +1.0 and minimization represent with -1.0.

In [24]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


We also define an Individual class, which will inherit the class list, and tell the DEAP creator module to assign FitnessMax as its fitness attribute.

In [25]:
creator.create("Individual", list, fitness=creator.FitnessMax)

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Now, with the Individual class defined, we use the toolbox of DEAP defined in the base module. We will use it to create a population and define our gene pool. All the objects that we will need from now onward—an individual, the population, the functions, the operators, and the arguments—are stored in a container called toolbox. We can add or remove content to/from the toolbox container using the register() and unregister() methods.

In [26]:
toolbox = base.Toolbox()
# Gene Pool
toolbox.register("attr_string", random.choice, \
               string.ascii_letters + string.digits )

Now that we have defined how the gene pool will be created, we create an individual and then a population by repeatedly using the Individual class. We will pass the class to the toolbox responsible for creating a N parameter , telling it how many genes to produce.

In [27]:
#Number of characters in word
# The word to be guessed
word = list('hello')
N = len(word)
# Initialize population
toolbox.register("individual", tools.initRepeat, \
         creator.Individual, toolbox.attr_string, N )
toolbox.register("population",tools.initRepeat, list,\
         toolbox.individual)

We define the fitness function. Note the comma in the return statement. This is because the fitness function in DEAP is returned as a tuple to allow multi-objective fitness functions.

In [28]:
def evalWord(individual, word):
    return sum(individual[i] == word[i] for i in\
            range(len(individual))),

Add the fitness function to the container. Also, add the crossover operator, mutation operator, and parent selector operator. You can see that, for this, we are using the register function. In the first statement, we register the fitness function that we have defined, along with the additional arguments it will take. The next statement registers the crossover operation; it specifies that here we are using a two-point crossover (cxTwoPoint). Next, we register the mutation operator; we choose the mutShuffleIndexes option, which shuffles the attributes of the input individual with a probability indpb=0.05. And finally, we define how the parents are selected; here, we have defined the method of selection as tournament selection with a tournament size of 3.

In [29]:
toolbox.register("evaluate", evalWord, word)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

Now we have all the ingredients to cook off, so we will write down the code of the genetic algorithm, which will perform the steps we mentioned earlier in a repetitive manner.

In [30]:
def main():
    random.seed(64)
    # create an initial population of 300 individuals 
    pop = toolbox.population(n=300)
    # CXPB is the crossover probability 
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.2
    print("Start of evolution")
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print(" Evaluated %i individuals" % len(pop))

    # Extracting all the fitnesses of individuals in a list
    fits = [ind.fitness.values[0] for ind in pop]
    # Variable keeping track of the number of generations
    g = 0

    # Begin the evolution
    while max(fits) < 5 and g < 1000:
        # A new generation
        g += 1
        print("-- Generation %i --" % g)

        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values
                for mutant in offspring:
                # mutate an individual with probability MUTPB
                    if random.random() < MUTPB:
                        toolbox.mutate(mutant)
                        del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print(" Evaluated %i individuals" % len(invalid_ind))

        # The population is entirely replaced by the offspring
        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print(" Min %s" % min(fits))
        print(" Max %s" % max(fits))
        print(" Avg %s" % mean)
        print(" Std %s" % std)

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (''.join(best_ind),\
    best_ind.fitness.values))

In [31]:
#Evaluate the Result
main()

Streaming output truncated to the last 5000 lines.
-- Generation 168 --
 Evaluated 300 individuals
 Min 0.0
 Max 3.0
 Avg 1.94
 Std 0.7635880215578731
-- Generation 169 --
 Evaluated 300 individuals
 Min 0.0
 Max 3.0
 Avg 1.78
 Std 0.7516204716034105
-- Generation 170 --
 Evaluated 300 individuals
 Min 0.0
 Max 3.0
 Avg 1.7766666666666666
 Std 0.7484131360566987
-- Generation 171 --
 Evaluated 300 individuals
 Min 0.0
 Max 3.0
 Avg 1.91
 Std 0.689371694612807
-- Generation 172 --
 Evaluated 300 individuals
 Min 0.0
 Max 3.0
 Avg 1.9366666666666668
 Std 0.7206401475231735
-- Generation 173 --
 Evaluated 300 individuals
 Min 0.0
 Max 3.0
 Avg 1.8966666666666667
 Std 0.7066273573972509
-- Generation 174 --
 Evaluated 300 individuals
 Min 0.0
 Max 3.0
 Avg 1.9266666666666667
 Std 0.6889283142840201
-- Generation 175 --
 Evaluated 300 individuals
 Min 0.0
 Max 3.0
 Avg 1.8766666666666667
 Std 0.6890492644861386
-- Generation 176 --
 Evaluated 300 individuals
 Min 0.0
 Max 3.0
 Avg 1.8566666

Now start to optimize the LSTM

## Optimizing LSTM with Genetic Algorithm for Wind Forecasting

The necessary modules are imported. This time, we are using Keras to implement the LSTM model.

In [32]:
!pip install bitstring
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as split

from keras.layers import LSTM, Input, Dense
from keras.models import Model

from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray

np.random.seed(1120)

The dataset we need for LSTM has to be time series data; we use the wind-power forecasting data from Kaggle (https://www.kaggle.com/c/GEF2012-wind-forecasting/data):

In [33]:
data = pd.read_csv('/content/drive/My Drive/optimization/train.csv')
data = np.reshape(np.array(data['wp1']),(len(data['wp1']),1))

train_data = data[0:17257]
test_data = data[17257:]

Define a function to prepare the dataset depending upon the chosen window_size.

In [34]:
def prepare_dataset(data, window_size):
    X, Y = np.empty((0,window_size)), np.empty((0))
    for i in range(len(data)-window_size-1):
        X = np.vstack([X,data[i:(i + window_size),0]])
        Y = np.append(Y,data[i + window_size,0])   
    X = np.reshape(X,(len(X),window_size,1))
    Y = np.reshape(Y,(len(Y),1))
    return X, Y

The train_evaluate function creates the LSTM network for a given individual and returns its RMSE value (fitness function).

In [35]:
def train_evaluate(ga_individual_solution):   
    # Decode GA solution to integer for window_size and num_units
    window_size_bits = BitArray(ga_individual_solution[0:6])
    num_units_bits = BitArray(ga_individual_solution[6:]) 
    window_size = window_size_bits.uint
    num_units = num_units_bits.uint
    print('\nWindow Size: ', window_size, ', Num of Units: ', num_units)
    
    # Return fitness score of 100 if window_size or num_unit is zero
    if window_size == 0 or num_units == 0:
        return 100, 
    
    # Segment the train_data based on new window_size; split into train and validation (80/20)
    X,Y = prepare_dataset(train_data,window_size)
    X_train, X_val, y_train, y_val = split(X, Y, test_size = 0.20, random_state = 1120)
    
    # Train LSTM model and predict on validation set
    inputs = Input(shape=(window_size,1))
    x = LSTM(num_units, input_shape=(window_size,1))(inputs)
    predictions = Dense(1, activation='linear')(x)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='adam',loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=5, batch_size=10,shuffle=True)
    y_pred = model.predict(X_val)
    
    # Calculate the RMSE score as fitness score for GA
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print('Validation RMSE: ', rmse,'\n')
    
    return rmse,

Next, we use DEAP tools to define Individual (again, since the chromosome is represented by a binary encoded string (10 bits), we use Bernoulli’s distribution), create the population, use ordered crossover, use mutShuffleIndexes mutation, and use the roulette wheel selection for selecting the parents.

In [36]:
population_size = 4
num_generations = 4
gene_length = 10

# As we are trying to minimize the RMSE score, that's why using -1.0. 
# In case, when you want to maximize accuracy for instance, use 1.0
creator.create('FitnessMax', base.Fitness, weights = (-1.0,))
creator.create('Individual', list , fitness = creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('binary', bernoulli.rvs, 0.5)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, n = gene_length)
toolbox.register('population', tools.initRepeat, list , toolbox.individual)

toolbox.register('mate', tools.cxOrdered)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.6)
toolbox.register('select', tools.selRoulette)
toolbox.register('evaluate', train_evaluate)

population = toolbox.population(n = population_size)
r = algorithms.eaSimple(population, toolbox, cxpb = 0.4, mutpb = 0.1, ngen = num_generations, verbose = False)

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)



Window Size:  36 , Num of Units:  2
Epoch 1/5
1378/1378 [==============================] - 5s 4ms/step - loss: 0.0142
Epoch 2/5
1378/1378 [==============================] - 5s 4ms/step - loss: 0.0080
Epoch 3/5
1378/1378 [==============================] - 5s 4ms/step - loss: 0.0067
Epoch 4/5
1378/1378 [==============================] - 5s 4ms/step - loss: 0.0061
Epoch 5/5
1378/1378 [==============================] - 5s 4ms/step - loss: 0.0059
Validation RMSE:  0.07657147567814025 


Window Size:  56 , Num of Units:  8
Epoch 1/5
1376/1376 [==============================] - 5s 4ms/step - loss: 0.0166
Epoch 2/5
1376/1376 [==============================] - 5s 4ms/step - loss: 0.0066
Epoch 3/5
1376/1376 [==============================] - 5s 4ms/step - loss: 0.0059
Epoch 4/5
1376/1376 [==============================] - 5s 4ms/step - loss: 0.0058
Epoch 5/5
1376/1376 [==============================] - 5s 4ms/step - loss: 0.0058
Validation RMSE:  0.07460835955711853 


Window Size:  60 , Num of

Print top N solutions - (1st only, for now)

In [37]:
best_individuals = tools.selBest(population,k = 1)
best_window_size = None
best_num_units = None

for bi in best_individuals:
    window_size_bits = BitArray(bi[0:6])
    num_units_bits = BitArray(bi[6:]) 
    best_window_size = window_size_bits.uint
    best_num_units = num_units_bits.uint
    print('\nWindow Size: ', best_window_size, ', Num of Units: ', best_num_units)


Window Size:  58 , Num of Units:  10


Train the model using best configuration on complete training set and make predictions on the test set.

In [38]:
X_train,y_train = prepare_dataset(train_data,best_window_size)
X_test, y_test = prepare_dataset(test_data,best_window_size)

inputs = Input(shape=(best_window_size,1))
x = LSTM(best_num_units, input_shape=(best_window_size,1))(inputs)
predictions = Dense(1, activation='linear')(x)
model = Model(inputs = inputs, outputs = predictions)
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(X_train, y_train, epochs=5, batch_size=10,shuffle=True)
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Test RMSE: ', rmse)

Epoch 1/5
1720/1720 [==============================] - 8s 4ms/step - loss: 0.0124
Epoch 2/5
1720/1720 [==============================] - 7s 4ms/step - loss: 0.0061
Epoch 3/5
1720/1720 [==============================] - 8s 4ms/step - loss: 0.0059
Epoch 4/5
1720/1720 [==============================] - 7s 4ms/step - loss: 0.0058
Epoch 5/5
1720/1720 [==============================] - 7s 4ms/step - loss: 0.0058
Test RMSE:  0.09254902559947434
